# Analyze Distribution

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

SITE = "googlemap"

df = pd.read_csv(f"../../database/reviews_{SITE}.csv")

# 텍스트 길이 계산
df["cleaned_content"] = df["text"].astype(str).str.strip()
df["review_len"] = df["cleaned_content"].str.len()

# 날짜 변환
df["date"] = pd.to_datetime(df["date"], format="%Y.%m.%d", errors="coerce")

# 저장 폴더
os.makedirs(f"plots/{SITE}", exist_ok=True)

## Rating dist.

In [10]:
plt.figure(figsize=(6, 4))
sns.countplot(x="rating", data=df)
plt.title(f"{SITE} - Rating Distribution")
plt.xlabel("Rating")
plt.ylabel("Count")
plt.savefig(f"plots/{SITE}/{SITE}_rating_dist.png", bbox_inches="tight")
plt.close()

## Review Length dist.

In [11]:
plt.figure(figsize=(6, 4))
sns.histplot(df["review_len"], bins=30, kde=True)
plt.title(f"{SITE} - Review Length Distribution")
plt.xlabel("Review Length")
plt.ylabel("Frequency")
plt.savefig(f"plots/{SITE}/{SITE}_reviewlen_dist.png", bbox_inches="tight")
plt.close()


## Date dist.

In [12]:
plt.figure(figsize=(10, 4))
df["date"].dropna().dt.to_period("M").value_counts().sort_index().plot()
plt.title(f"{SITE} - Monthly Review Count")
plt.xlabel("Month")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.savefig(f"plots/{SITE}/{SITE}_date_dist.png", bbox_inches="tight")
plt.close()


# Outlier Detection

In [14]:
today = pd.Timestamp.today()

# 1. 형식 이상치 (별점 숫자 변환 실패)
rating_str = df["rating"].astype(str)
rating_numeric = pd.to_numeric(rating_str, errors="coerce")
invalid_format_rating = df[rating_numeric.isna()]

# 2. 범위 이상치 (1~5 외)
valid_numeric_rating = df[rating_numeric.notna()].copy()
valid_numeric_rating["rating"] = rating_numeric.dropna()
out_of_range_rating = valid_numeric_rating[
    (valid_numeric_rating["rating"] < 1) | (valid_numeric_rating["rating"] > 5)
]

# 3. 텍스트 길이 이상치
too_short = df[df["review_len"] < 3]
too_short_or_empty = df[
    (df["cleaned_content"] == "") | (df["review_len"] < 3)
]
too_long = df[df["review_len"] > 1800]

# 4. 날짜 이상치
too_old = df[df["date"] < "2015-01-01"]
future_date = df[df["date"] > today]
non_parsable_date = df[df["date"].isna()]

# Outlier Count 기록
outlier_counts = {
    "별점 형식 오류": len(invalid_format_rating),
    "별점 범위 이상치": len(out_of_range_rating),
    "너무 짧은 리뷰": len(too_short_or_empty),
    "너무 긴 리뷰": len(too_long),
    "너무 오래된 날짜": len(too_old),
    "미래 날짜": len(future_date),
    "날짜 형식 오류": len(non_parsable_date),
}


# Missing value Detection

In [17]:
missing_counts = df[["rating", "date", "text"]].isna().sum().to_dict()

# 출력
print("──────────── Outlier Report ────────────")
for k, v in outlier_counts.items():
    print(f"[{k:<10}] {v}개")

print("\n──────────── Missing Value Report ──────")
for k, v in missing_counts.items():
    print(f"[{k:<10}] {v}개")
print("────────────────────────────────────────")

──────────── Outlier Report ────────────
[별점 형식 오류  ] 0개
[별점 범위 이상치 ] 0개
[너무 짧은 리뷰  ] 0개
[너무 긴 리뷰   ] 0개
[너무 오래된 날짜 ] 0개
[미래 날짜     ] 0개
[날짜 형식 오류  ] 38개

──────────── Missing Value Report ──────
[rating    ] 0개
[date      ] 38개
[text      ] 0개
────────────────────────────────────────
